In [3]:
import numpy as np
np.random.seed(2020)
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import alpino
from nltk.tag import UnigramTagger
from random import shuffle
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from datetime import datetime
from statistics import mean

In [4]:
DF = pd.read_json('ELK.json', encoding='utf-8-sig', lines=True)
print(DF.shape)
DF.groupby('shop').count()

(324324, 8)


,globalId,authorAgeRange,submissionDateTime,cluster,rating,text,title
shop,,,,,,,
Audio en Televisie,50387,32751,50387,50387,50387,50387,50384
Camera en Accessoires,26524,16804,26524,26524,26524,26524,26524
Computer Accessoires,54291,35934,54291,54291,54291,54291,54290
Computer Hardware,10476,7468,10476,10476,10476,10476,10476
Groot Huishoudelijk,15792,11629,15792,15792,15792,15792,15792
Klein Huishoudelijk,84583,63277,84583,84583,84583,84583,84583
Mobiele Hardware en Wearables,30238,20715,30238,30238,30238,30238,30236
Telefoon en Tablet Accessoires,52033,34582,52033,52033,52033,52033,52033


In [6]:
DF['submissionDateTime'] = pd.to_datetime(DF['submissionDateTime'], yearfirst=True, utc=True)
DF = DF[['globalId', 'authorAgeRange', 'submissionDateTime', 'shop', 'cluster', 'rating', 'text', 'title']]

**SORT BY MONTH AND POLARITY** (only takes into accounts 5 or 1 star reviews)

In [8]:
# MSB_january = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'January'].to_numpy()]
january_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'January')&(DF['rating'] ==5)].to_numpy()]
january_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'January')&(DF['rating'] ==1)].to_numpy()]

# MSB_february = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'February'].to_numpy()]
february_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'February')&(DF['rating'] ==5)].to_numpy()]
february_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'February')&(DF['rating'] ==1)].to_numpy()]

# MSB_march = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'March'].to_numpy()]
march_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'March')&(DF['rating'] ==5)].to_numpy()]
march_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'March')&(DF['rating'] ==1)].to_numpy()]

# MSB_april = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'April'].to_numpy()]
april_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'April')&(DF['rating'] ==5)].to_numpy()]
april_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'April')&(DF['rating'] ==1)].to_numpy()]

# MSB_may = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'May'].to_numpy()]
may_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'May')&(DF['rating'] ==5)].to_numpy()]
may_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'May')&(DF['rating'] ==1)].to_numpy()]

# MSB_june = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'June'].to_numpy()]
june_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'June')&(DF['rating'] ==5)].to_numpy()]
june_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'June')&(DF['rating'] ==1)].to_numpy()]

# MSB_july = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'July'].to_numpy()]
july_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'July')&(DF['rating'] ==5)].to_numpy()]
july_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'July')&(DF['rating'] ==1)].to_numpy()]

# MSB_august = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'August'].to_numpy()]
august_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'August')&(DF['rating'] ==5)].to_numpy()]
august_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'August')&(DF['rating'] ==1)].to_numpy()]

# MSB_september = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'September'].to_numpy()]
september_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'September')&(DF['rating'] ==5)].to_numpy()]
september_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'September')&(DF['rating'] ==1)].to_numpy()]

# MSB_october = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'October'].to_numpy()]
october_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'October')&(DF['rating'] ==5)].to_numpy()]
october_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'October')&(DF['rating'] ==1)].to_numpy()]

# MSB_november = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'November'].to_numpy()]
november_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'November')&(DF['rating'] ==5)].to_numpy()]
november_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'November')&(DF['rating'] ==1)].to_numpy()]

# MSB_december = [x for x in DF.loc[DF['submissionDateTime'].dt.strftime('%B') == 'December'].to_numpy()]
december_pos = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'December')&(DF['rating'] ==5)].to_numpy()]
december_neg = [x for x in DF.loc[(DF['submissionDateTime'].dt.strftime('%B') == 'December')&(DF['rating'] ==1)].to_numpy()]

pos = [x for x in DF.loc[DF['rating'] ==5].to_numpy()]
neg = [x for x in DF.loc[DF['rating'] ==1].to_numpy()]

In [11]:
def main_func(XX):
    obj_analysis = XX
    texts = [item[6] for item in obj_analysis]
    labels = [item[5] for item in obj_analysis]
    labels = [1 if i > 3 else 0 for i in labels]
    def preprocessing(text, pos_tags=False):
        # tokenize into words
        tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]   
        stop = stopwords.words('dutch')  # remove stopwords
        stop.extend(['mee', 'het', 'erg', 'wel', 'zeer', 'wij', 'ik', 'ben', 'jij', 'jullie', 'echt', 
                           'wel', 'heel', 'erg', 'mee', 'dus', 'maar']) #extra stopwords     
        tokens = [token for token in tokens if token not in stop]
        tokens = [word for word in tokens if len(word) >= 3] # remove words less than three characters
        tokens = [word.lower() for word in tokens] # remove capitalization
        lmtzr = WordNetLemmatizer()
        tokens = [lmtzr.lemmatize(word) for word in tokens] # lemmatizing
        preprocessed_text= ' '.join(tokens)
        return preprocessed_text

    def preprocesser(list_of_texts): # turning lists into preprocessed lists
        new = []
        for item in list_of_texts:
            new += [preprocessing(item)]
        return new
    
    corpus = [r for r in preprocesser(texts)]

    #selecting documents with highest subjectivity, based on relative number of present adjectives
    training_corpus = alpino.tagged_sents() # dutch training corpus
    unitagger = UnigramTagger(training_corpus)
    pos_tag = unitagger.tag
    corpus_POS = []  
    for item in corpus:
        tokens = nltk.word_tokenize(item)
        pos_item = pos_tag(tokens)
        new_sent = ['_'.join(t) for t in pos_item if t[1] is not None]
        corpus_POS += [new_sent]
    corpus_POS
    
    ADJ_list = [(1+i.count('adj')) / len(i) for i in corpus_POS] # relative number of adjectives per document
    ADJ_indices = [i for i in range(len(ADJ_list)) if ADJ_list[i] > mean(ADJ_list)] # highly subjective document indexes  

    corpus = [corpus[i] for i in ADJ_indices]
    shuffle(corpus)
    
    vectorizer = CountVectorizer(min_df=4, max_df=0.3, ngram_range=(1,1))
    X = vectorizer.fit_transform(corpus)
    X = pd.DataFrame(X.todense(),columns=vectorizer.get_feature_names())
    
    lda_model = LatentDirichletAllocation(n_components=3, max_iter=10, learning_decay=0.7)
    doc_topic = lda_model.fit_transform(X)

    def display_topics(model, feature_names, no_top_words):
        with open('results_topics_ELK.txt', 'a', encoding='utf-8-sig') as file: 
            file.write(str(datetime.now()) + '\n')
            if len(np.unique(np.array(obj_analysis)[:,3])) == 1:  #adds store specific
                file.write(str(obj_analysis[0][3]) + ' | ') 
            file.write(str(obj_analysis[0][4])
                   + str(' | Positive | ' + str(obj_analysis[0][2]) + '\n\n' if obj_analysis[0][5] > 3 
                     else ' | Negative | '+ str(obj_analysis[0][2]) + '\n\n'))

            for topic_idx, topic in enumerate(model.components_):
                print("Topic %d:" % (topic_idx)," ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]) + '\n')
                file.write("Topic %d: " % (topic_idx))    
                file.write(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])+'\n')              
        
            # Log Likelyhood: Higher the better
            print("Log Likelihood: ", lda_model.score(X))
            file.write("Log Likelihood: " + str(lda_model.score(X))+ ' \n')
            # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
            print("Perplexity: ", lda_model.perplexity(X))
            file.write("Perplexity: " + str(lda_model.perplexity(X))+ ' \n')
            # See model parameters
            print(lda_model.get_params())
            file.write(str(lda_model.get_params())+ ' \n')
            file.write('\n\n\n')
            
    no_top_words = 20
    feature_names = vectorizer.get_feature_names()
    display_topics(lda_model, feature_names, no_top_words)        

In [12]:
main_func(december_neg)

Topic 0: scherm werkt het past hoesje helaas telefoon goed zit waardoor helemaal ook gsm alleen blijft protector gaat terug mooi zitten

Topic 1: product het goed hoesje dit gebruik kwaliteit telefoon slecht bol jammer los apparaat echter zit zitten toestel com binnen prijs

Topic 2: kabel geluid slecht geld kwaliteit goed keer slechte jaar het doet gekocht kapot binnen gebruiken maakt kopen helaas niet apparaat

Log Likelihood:  -147642.30935581704
Perplexity:  956.4739013795825
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'batch', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 3, 'n_jobs': None, 'perp_tol': 0.1, 'random_state': None, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [10]:
my_list = [january_pos, january_neg,
          february_pos, february_neg,
          march_pos, march_neg,
          april_pos, april_neg,
          may_pos, may_neg,
           june_pos, june_neg,
           july_pos, july_neg,
           august_pos, august_neg,
           september_pos, september_neg,
           october_pos, october_neg, 
           november_pos, november_neg,
           december_pos, december_neg
          ] 

In [11]:
for item in my_list:
    main_func(item)

Topic 0: apparaat het makkelijk gebruik snel geluid super maken stofzuiger handig aanrader blij tevreden mooi prima koffie goede een zeker schoon

Topic 1: het telefoon werkt goede prima prijs snel gebruik via laptop mooi perfect kwaliteit bol makkelijk scherm super geluid hoesje aanrader

Topic 2: het gebruik gaat mooi muis camera vind zeker fijn hand goede lang handig makkelijk geluid zit werkt snel foto prima

Log Likelihood:  -3807258.4324339516
Perplexity:  2067.9649602625723
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'batch', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 3, 'n_jobs': None, 'perp_tol': 0.1, 'random_state': None, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}
Topic 0: hoesje goed het scherm telefoon waardoor zit past product los screenprotector kwaliteit zitten iphone protector slecht plastic komt geld ziet

Topic 1

Topic 0: het gebruik makkelijk toestel snel scherm telefoon mooi maken apparaat handig zit super werkt zeker tevreden een dit ook hoesje

Topic 1: apparaat gebruik het snel foto bol camera goede makkelijk werkt prijs tevreden super com prima kwaliteit koffie product gaat weer

Topic 2: geluid het goede gebruik prijs mooi super prima kwaliteit zeker werkt makkelijk gaat zit muziek muis stofzuiger aanrader laptop via

Log Likelihood:  -2836676.2175864005
Perplexity:  2010.3619229274682
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'batch', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 3, 'n_jobs': None, 'perp_tol': 0.1, 'random_state': None, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}
Topic 0: telefoon werkt weer gekocht batterij goed gaat toestel slecht keer het jaar app gewoon helemaal bol kopen alleen zit opladen

Topic 1: apparaat het

Topic 0: apparaat het gebruik makkelijk snel koffie stofzuiger maken handig mooi gaat super schoon ook tevreden zeker maakt lang fijn vind

Topic 1: geluid het goede telefoon mooi prijs werkt prima kwaliteit via makkelijk super hoesje bol zit perfect zeker com tevreden gebruik

Topic 2: het gebruik snel prijs laptop werkt muis goede mooi toestel kwaliteit gaat zeker handig vind prima makkelijk aanrader super fijn

Log Likelihood:  -2879693.4960708264
Perplexity:  1948.4547933333893
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'batch', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 3, 'n_jobs': None, 'perp_tol': 0.1, 'random_state': None, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}
Topic 0: bol werkt com product kabel terug weer apparaat toestel het goed printer staat via alleen moeten artikel werkte helaas dit

Topic 1: kwaliteit het ap